In [ ]:
import pandas as pd
data = pd.read_csv("matala2_cosmetics_2019-Nov.csv")



## quastion 1

In [ ]:
data['event_time'] = pd.to_datetime(data['event_time'])
data['duration_to_next_event'] = data.groupby('user_session')['event_time'].apply(lambda x: x.shift(-1) - x)
data['duration_to_next_event'] = data['duration_to_next_event'].astype('timedelta64[s]').fillna(0)
data.head()

## quastion 2

In [ ]:
data['funnel_number'] = data.groupby('user_id')['event_time'].apply(
    lambda x: (x - x.shift(1)).dt.days.gt(5).cumsum()+1)

## quastion 3

In [ ]:
data['index_in_funnel'] = data.groupby(['user_id' , 'funnel_number'])['user_session'].rank(method='first')
data

## quastion 4

In [ ]:

import re
data['price'] = data['price'].apply(lambda x: re.search('[0-9]+\.+[0-9]+' , x).group()).astype(float)
data

## quastion 5

In [ ]:
_=data.event_type.value_counts().plot.bar()

## quastion 6


In [ ]:
session_data = data[['user_id', 'user_session' , 'funnel_number' , 'index_in_funnel']].groupby('user_session').first()
session_data['event_count'] = data.groupby('user_session').size()
session_data['session_duration'] = data[['duration_to_next_event' , 'user_session']].groupby('user_session').sum()
session_data['list_of_viewed'] = data[data['event_type'] == 'view'].groupby('user_session')['product_id'].apply(list)
session_data['list_of_added_to_cart'] = data[data['event_type'] == 'cart'].groupby('user_session')['product_id'].apply(list)
session_data['list_of_purchased'] = data[data['event_type'] == 'purchase'].groupby('user_session')['product_id'].apply(list)
session_data